In [ ]:
import pickle
import time
from selenium import webdriver

driver = webdriver.Chrome()

def guardar_cookies():
    driver.get("https://www.linkedin.com/login")
    print("Por favor, logueate manualmente y resuelve el CAPTCHA...")
    
    # Espera 60 segundos para que te de tiempo de entrar
    time.sleep(60) 
    
    # Guardamos las cookies en un archivo .pkl (es más eficiente que JSON para objetos de Selenium)
    with open("linkedin_cookies.pkl", "wb") as file:
        pickle.dump(driver.get_cookies(), file)
    
    print("¡Cookies guardadas con éxito!")
    driver.quit()

guardar_cookies()

Por favor, logueate manualmente y resuelve el CAPTCHA...
¡Cookies guardadas con éxito!


In [9]:
import pickle
import time
import random
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Configuración inicial
options = webdriver.ChromeOptions()
# options.add_argument("--incognito") # Descomenta si prefieres incógnito
driver = webdriver.Chrome(options=options)

def cargar_sesion_con_cookies():
    print("Intentando cargar sesión mediante cookies...")
    driver.get("https://www.linkedin.com")
    time.sleep(3)
    try:
        with open("linkedin_cookies.pkl", "rb") as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                # Eliminamos el expiry para evitar conflictos de fecha
                if 'expiry' in cookie:
                    del cookie['expiry']
                driver.add_cookie(cookie)
        driver.refresh()
        time.sleep(5)
    except FileNotFoundError:
        print("⚠️ Archivo de cookies no encontrado.")

def esta_logueado():
    try:
        # Buscamos el nav global o la caja de búsqueda
        driver.find_element(By.CLASS_NAME, "global-nav__content")
        return True
    except NoSuchElementException:
        return False

def realizar_scraping_anonimo(tema):
    # 1. Búsqueda inicial
    search_url = f"https://www.linkedin.com/search/results/content/?keywords={tema}"
    driver.get(search_url)
    print(f"Buscando posts sobre: {tema}")
    time.sleep(8) 

    with open('comentarios_ia_anonimos.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Comentario"])

        # 2. Localizar los posts (usando el rol de lista del DOM que pasaste)
        posts = driver.find_elements(By.XPATH, "//div[contains(@data-view-tracking-scope, 'FeedUpdateServedEvent')]")
        print(f"Posts detectados: {len(posts)}")

        conteo_total = 0
        for i, post in enumerate(posts[:10]): # Limitamos a 10 para asegurar calidad
            try:
                # Scroll para activar el post
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", post)
                time.sleep(2)

                # PASO A: Click en el botón de conteo (ej. "48 comentarios")
                # Usamos el data-view-name exacto que encontraste
                try:
                    boton_conteo = post.find_element(By.CSS_SELECTOR, "div[data-view-name='feed-comment-count']")
                    driver.execute_script("arguments[0].click();", boton_conteo)
                    print(f"Post {i+1}: Abriendo sección de comentarios...")
                    time.sleep(4)
                except:
                    print(f"Post {i+1}: No tiene comentarios visibles o botón de conteo.")
                    continue

                # PASO B: Bucle para dar click en "Cargar más comentarios" DENTRO de este post
                print(f"Post {i+1}: Expandiendo hilos...")
                while True:
                    try:
                        # Buscamos el botón "Cargar más" específico de esta sección
                        boton_mas = post.find_element(By.CSS_SELECTOR, "button[data-view-name='more-comments']")
                        driver.execute_script("arguments[0].click();", boton_mas)
                        time.sleep(random.uniform(2.5, 4.0))
                    except:
                        # Si ya no hay botón de "Cargar más", salimos del bucle
                        break

                # PASO C: Extraer todos los comentarios del post ya expandido
                # Usamos el selector data-testid que verificamos antes
                comentarios_html = post.find_elements(By.CSS_SELECTOR, "span[data-testid='expandable-text-box']")
                
                for c in comentarios_html:
                    texto = c.text.strip()
                    # Filtro de seguridad e integridad
                    if texto and len(texto) > 5:
                        writer.writerow([texto.replace('\n', ' ')])
                        conteo_total += 1
                
                print(f"Post {i+1}: Procesado con éxito.")

            except Exception as e:
                print(f"Error en post {i+1}: {e}")
                continue

    print(f"\n✅ SCRAPING FINALIZADO.")
    print(f"📊 Total recolectado: {conteo_total} comentarios.")


def buscar_y_extraer_seguro(tema):
    intentos_fallidos = 0
    max_intentos = 2
    
    cargar_sesion_con_cookies()
    
    while True:
        if esta_logueado():
            print("✅ Sesión activa. Iniciando búsqueda...")
            realizar_scraping_anonimo(tema)
            break
        else:
            intentos_fallidos += 1
            print(f"❌ Intento {intentos_fallidos} fallido.")
            
            if intentos_fallidos < max_intentos:
                print("Esperando 1 minuto para reintentar automáticamente...")
                time.sleep(60)
                cargar_sesion_con_cookies()
            else:
                print("\nSISTEMA BLOQUEADO: Por favor, inicia sesión MANUALMENTE.")
                while not esta_logueado():
                    time.sleep(5)
                
                print("✅ ¡Acceso manual detectado!")
                with open("linkedin_cookies.pkl", "wb") as file:
                    pickle.dump(driver.get_cookies(), file)
                continue

# Ejecución
if __name__ == "__main__":
    buscar_y_extraer_seguro("IA desarrollo software")

Intentando cargar sesión mediante cookies...
✅ Sesión activa. Iniciando búsqueda...
Buscando posts sobre: IA desarrollo software
Posts detectados: 10
Post 1: Abriendo sección de comentarios...
Post 1: Expandiendo hilos...
Post 1: Procesado con éxito.
Post 2: No tiene comentarios visibles o botón de conteo.
Post 3: Abriendo sección de comentarios...
Post 3: Expandiendo hilos...
Post 3: Procesado con éxito.
Post 4: No tiene comentarios visibles o botón de conteo.
Post 5: Abriendo sección de comentarios...
Post 5: Expandiendo hilos...
Post 5: Procesado con éxito.
Post 6: No tiene comentarios visibles o botón de conteo.
Post 7: Abriendo sección de comentarios...
Post 7: Expandiendo hilos...
Post 7: Procesado con éxito.
Post 8: No tiene comentarios visibles o botón de conteo.
Post 9: Abriendo sección de comentarios...
Post 9: Expandiendo hilos...
Post 9: Procesado con éxito.
Post 10: No tiene comentarios visibles o botón de conteo.

✅ SCRAPING FINALIZADO.
📊 Total recolectado: 56 comentarios